In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import torch

In [3]:
!git clone https://github.com/Anil-Banjade/Nepali_Multi_Modal.git


Cloning into 'Nepali_Multi_Modal'...
remote: Enumerating objects: 385, done.
remote: Counting objects: 100% (117/117), done.
remote: Compressing objects: 100% (71/71), done.
remote: Total 385 (delta 64), reused 80 (delta 33), pack-reused 268 (from 1)
Receiving objects: 100% (385/385), 1.93 MiB | 4.91 MiB/s, done.
Resolving deltas: 100% (221/221), done.


In [4]:
!git fetch origin
!git reset --hard origin/main

fatal: not a git repository (or any of the parent directories): .git
fatal: not a git repository (or any of the parent directories): .git


In [5]:
%cd Nepali_Multi_Modal

/content/Nepali_Multi_Modal


In [6]:
%%capture
!pip install -r requirements.txt

In [7]:
from src.multimodal_text_generation.main import main

Using device: cuda


In [11]:
if __name__ == "__main__":
    model=main()
    torch.save(model.state_dict(), '/content/drive/MyDrive/Minor_project/autoregressive_model.pt')


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/652 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/547k [00:00<?, ?B/s]

tf_model.h5:   0%|          | 0.00/534M [00:00<?, ?B/s]

All TF 2.0 model weights were used when initializing BertModel.

All the weights of BertModel were initialized from the TF 2.0 model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use BertModel for predictions without further training.
/content/Nepali_Multi_Modal/src/multimodal_text_generation/main.py:17: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serializati

Epoch [0/5], Batch [0/8090], Loss: 10.3267
Epoch [0/5], Batch [1000/8090], Loss: 2.2536
Epoch [0/5], Batch [2000/8090], Loss: 1.7712
Epoch [0/5], Batch [3000/8090], Loss: 1.8162
Epoch [0/5], Batch [4000/8090], Loss: 1.9098
Epoch [0/5], Batch [5000/8090], Loss: 1.0151
Epoch [0/5], Batch [6000/8090], Loss: 1.1711
Epoch [0/5], Batch [7000/8090], Loss: 0.8860
Epoch [0/5], Batch [8000/8090], Loss: 0.8119
Epoch [0/5], Average Loss: 1.7745
Epoch [1/5], Batch [0/8090], Loss: 1.2106
Epoch [1/5], Batch [1000/8090], Loss: 1.0688
Epoch [1/5], Batch [2000/8090], Loss: 0.8887
Epoch [1/5], Batch [3000/8090], Loss: 0.6236
Epoch [1/5], Batch [4000/8090], Loss: 1.0020
Epoch [1/5], Batch [5000/8090], Loss: 0.9088
Epoch [1/5], Batch [6000/8090], Loss: 0.5364
Epoch [1/5], Batch [7000/8090], Loss: 0.5732
Epoch [1/5], Batch [8000/8090], Loss: 0.6560
Epoch [1/5], Average Loss: 0.9076
Epoch [2/5], Batch [0/8090], Loss: 0.8503
Epoch [2/5], Batch [1000/8090], Loss: 0.6703
Epoch [2/5], Batch [2000/8090], Loss: 0.

In [8]:
from src.multimodal_text_generation.main import test

In [9]:
test()

Loading model...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/652 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/547k [00:00<?, ?B/s]


Attempt 1/3
Image embedding shape: torch.Size([1, 1, 768])


tf_model.h5:   0%|          | 0.00/534M [00:00<?, ?B/s]

All TF 2.0 model weights were used when initializing BertModel.

All the weights of BertModel were initialized from the TF 2.0 model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use BertModel for predictions without further training.


BOS embedding shape: torch.Size([1, 1, 768])
Current embeddings shape: torch.Size([1, 2, 768])
Starting generation loop...
Generated token 0: 75 -> s
Partial caption: s
Generated token 1: 1342 -> ##a
Generated token 2: 1342 -> ##a
Partial caption: saa
Generated token 3: 1922 -> मा
Generated token 4: 1342 -> ##a
Partial caption: saa माa
Generated token 5: 1342 -> ##a
Generated token 6: 1491 -> ##t
Partial caption: saa माaat
Generated token 7: 2 -> [SEP]
End token generated after meaningful sequence
Final generated caption: saa माaat
Original: कालो कुकुर र दाग भएको कुकुर लडिरहेका छन्
Generated: saa माaat
